<a href="https://colab.research.google.com/github/sergioar3143/AprendizajeProfundo/blob/main/Clasificacion_grupo_etario_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.signal import correlate2d, convolve2d
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import PIL
import copy
from torch import nn
from torch.optim import Adam
from torch.optim import SGD
from sklearn.model_selection import train_test_split
import torchvision as tv

##Audio
from torchaudio.datasets import SPEECHCOMMANDS
from tqdm.auto import trange
import collections
import torchaudio.transforms as T
import librosa
import random
import IPython as ip

# Por reproducibilidad
torch.manual_seed(42)
np.random.seed(42)

!gdown --id 1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86
!gdown --id 1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH
!gdown --id 1wOdja-ezstMEp81tX1a-EYkFebev4h7D
!rm -r "/content/FairFace"
!unzip "/content/fairface-img-margin025-trainval.zip" -d "/content/FairFace"